### Image Noise Reduction
Implement a model that can perform image noise reduction, enhancing the quality of images by reducing noise while preserving important features.

### Objective
Develop and train models that effectively reduce noise in images, improving overall clarity and visual quality.

### Tasks
- Data Cleaning & Transformation: Preprocessing the provided dataset of clean images to generate noisy images for model training.
- Feature Engineering: Analyze noise patterns and apply transformations to optimize the dataset.
- Encoding of images to ensure they are suitable for processing by the models, utilizing libraries such as `OpenCV` for image manipulation.

### Link to the dataset
https://drive.google.com/file/d/1yqLUYqU8_elMhrk1Ebe6p-RdHgmixuta/view?usp=sharing